In [1]:
import gym_battleship
import gymnasium as gym
import numpy as np
import pandas as pd
from stable_baselines3 import DQN

Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
Users of this version of Gym should be able to simply replace 'import gym' with 'import gymnasium as gym' in the vast majority of cases.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
env = gym.make('Battleship-v0', board_size=(10, 10))
print(env)
print(dir(env))
env.reset()

<OrderEnforcing<PassiveEnvChecker<BattleshipEnv<Battleship-v0>>>>
['__annotations__', '__class__', '__class_getitem__', '__delattr__', '__dict__', '__dir__', '__doc__', '__enter__', '__eq__', '__exit__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__orig_bases__', '__parameters__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_action_space', '_cached_spec', '_disable_render_order_enforcing', '_has_reset', '_is_protocol', '_metadata', '_np_random', '_np_random_seed', '_observation_space', '_saved_kwargs', 'action_space', 'class_name', 'close', 'env', 'get_wrapper_attr', 'has_reset', 'has_wrapper_attr', 'metadata', 'np_random', 'np_random_seed', 'observation_space', 'render', 'render_mode', 'reset', 'set_wrapper_attr', 'spec', 'step', 'unwrapped', 'wrapper_spec']


(array([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],
 
        [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.

In [5]:
import torch as th
import torch.nn as nn
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor
from stable_baselines3.dqn import CnnPolicy

class BattleshipCNN(BaseFeaturesExtractor):
    def __init__(self, observation_space, features_dim=128):
        super().__init__(observation_space, features_dim)

        n_channels = observation_space.shape[0]  

        self.cnn = nn.Sequential(
            nn.Conv2d(n_channels, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Flatten()
        )

        with th.no_grad():
            sample = th.zeros((1, n_channels, 10, 10))
            sample_out = self.cnn(sample)
            conv_output_dim = sample_out.shape[1]

        self.linear = nn.Sequential(
            nn.Linear(conv_output_dim, features_dim),
            nn.ReLU()
        )

    def forward(self, obs):
        return self.linear(self.cnn(obs))


In [6]:
policy_kwargs = dict(
    features_extractor_class=BattleshipCNN,
    features_extractor_kwargs=dict(features_dim=128),
)

model = DQN(
    "CnnPolicy",
    env,
    policy_kwargs=policy_kwargs,
    learning_rate=1e-4,
    verbose=1,
)


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/stable_baselines3/common/buffers.py:242: UserWarning: This system does not have apparently enough memory to store the complete replay buffer 2.42GB > 1.33GB
  warnings.warn(


In [7]:
model.learn(total_timesteps=200000, log_interval=10)
model.save("battleship_dqn")

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 100      |
|    ep_rew_mean      | -31.5    |
|    exploration_rate | 0.952    |
| time/               |          |
|    episodes         | 10       |
|    fps              | 286      |
|    time_elapsed     | 3        |
|    total_timesteps  | 1000     |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.223    |
|    n_updates        | 224      |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 100      |
|    ep_rew_mean      | -33.1    |
|    exploration_rate | 0.905    |
| time/               |          |
|    episodes         | 20       |
|    fps              | 304      |
|    time_elapsed     | 6        |
|    total_timesteps  | 2000     |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.301    |
|    n_updates      

In [8]:
obs, info = env.reset()
while True:
    action, _states = model.predict(obs, deterministic=True)
    print("Model chose action: ", action)
    # print(type(action))
    # print(dir(action))
    # print(int(action))
    obs, reward, terminated, truncated, info = env.step(int(action))
    env.render()
    if terminated or truncated:
        break
    input()
    


Model chose action:  78


,A,B,C,D,E,F,G,H,I,J
1,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜
2,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜
3,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜
4,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜
5,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜
6,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜
7,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜
8,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜
9,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⚫,⬜,⬜
10,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜


Model chose action:  15


,A,B,C,D,E,F,G,H,I,J
1,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜
2,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜
3,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜
4,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜
5,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜
6,⬜,⚫,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜
7,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜
8,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜
9,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⚫,⬜,⬜
10,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜


Model chose action:  86


...

Model chose action:  62


,A,B,C,D,E,F,G,H,I,J
1,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜
2,⚫,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜
3,⬜,⬜,⬜,⬜,⬜,⬜,⚫,⬜,⬜,⬜
4,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⚫,⬜
5,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜
6,⬜,⚫,⬜,⬜,⬜,⬜,❌,⬜,⬜,⬜
7,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,❌,⬜
8,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜
9,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⚫,⬜,⬜
10,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜


Model chose action:  62


,A,B,C,D,E,F,G,H,I,J
1,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜
2,⚫,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜
3,⬜,⬜,⬜,⬜,⬜,⬜,⚫,⬜,⬜,⬜
4,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⚫,⬜
5,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜
6,⬜,⚫,⬜,⬜,⬜,⬜,❌,⬜,⬜,⬜
7,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,❌,⬜
8,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜
9,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⚫,⬜,⬜
10,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜


KeyboardInterrupt: Interrupted by user